# Eat Safe, Love

## Notebook Set Up

In [74]:
# Import dependencies
from pymongo import MongoClient
import pandas as pd
from pprint import pprint
import json 
import requests

In [75]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [76]:
# assign the uk_food database to a variable name
db = mongo['uk_foods']

In [77]:
# review the collections in our database
print(db.list_collection_names())

['establishments']


In [78]:
# assign the collection to a variable
establishments = db['establishments']

## Part 3: Exploratory Analysis
Unless otherwise stated, for each question: 
* Use `count_documents` to display the number of documents contained in the result.
* Display the first document in the results using `pprint`.
* Convert the result to a Pandas DataFrame, print the number of rows in the DataFrame, and display the first 10 rows.

### 1. Which establishments have a hygiene score equal to 20?

In [79]:
# Find the establishments with a hygiene score of 20
query = {'scores.Hygiene': 20}
fields = {'BusinessName': 1, 'scores.Hygiene': 1}
hygiene_scores = list(establishments.find(query,fields))

# Use count_documents to display the number of documents in the result
print("Number of establishments with a hygiene score of 20 are:", establishments.count_documents(query))

# Display the first document in the results using pprint
for hscores in range(1):
    pprint(hygiene_scores[hscores])

Number of establishments with a hygiene score of 20 are: 41
{'BusinessName': 'The Chase Rest Home',
 '_id': ObjectId('63d1ef270eddd0949a7badff'),
 'scores': {'Hygiene': 20}}


In [80]:
# Convert the result to a Pandas DataFrame
cleanly = pd.DataFrame(hygiene_scores)
# Display the number of rows in the DataFrame
cleanly.count()
# Display the first 10 rows of the DataFrame
cleanly.head(10)

,_id,BusinessName,scores
0,63d1ef270eddd0949a7badff,The Chase Rest Home,{'Hygiene': 20}
1,63d1ef270eddd0949a7bb181,Brenalwood,{'Hygiene': 20}
2,63d1ef270eddd0949a7bb48a,Melrose Hotel,{'Hygiene': 20}
3,63d1ef270eddd0949a7bb673,Seaford Pizza,{'Hygiene': 20}
4,63d1ef270eddd0949a7bb684,Golden Palace,{'Hygiene': 20}
5,63d1ef270eddd0949a7bc031,Ashby's Butchers,{'Hygiene': 20}
6,63d1ef270eddd0949a7bc249,South Sea Express Cuisine,{'Hygiene': 20}
7,63d1ef280eddd0949a7bd775,Golden Palace,{'Hygiene': 20}
8,63d1ef280eddd0949a7bdbb1,The Tulip Tree,{'Hygiene': 20}
9,63d1ef280eddd0949a7be3c5,F & S,{'Hygiene': 20}


### 2. Which establishments in London have a `RatingValue` greater than or equal to 4?

In [82]:
# Going to convert Rating value to a int before querying 
db.establishments.update_many({},{'$set': 
    {'$toInt': "$RatingValue"}}
)
# Find the establishments with London as the Local Authority and has a RatingValue greater than or equal to 4.
#query = 

# Use count_documents to display the number of documents in the result

# Display the first document in the results using pprint


WriteError: The dollar ($) prefixed field '$toInt' in '$toInt' is not allowed in the context of an update's replacement document. Consider using an aggregation pipeline with $replaceWith.

In [ ]:
# Convert the result to a Pandas DataFrame

# Display the number of rows in the DataFrame

# Display the first 10 rows of the DataFrame


### 3. What are the top 5 establishments with a `RatingValue` rating value of '5', sorted by lowest hygiene score, nearest to the new restaurant added, "Penang Flavours"?

In [ ]:
# Search within 0.01 degree on either side of the latitude and longitude.
# Rating value must equal 5
# Sort by hygiene score

degree_search = 0.01
latitude = 
longitude = 

query = 
sort =  

# Print the results


In [ ]:
# Convert result to Pandas DataFrame


### 4. How many establishments in each Local Authority area have a hygiene score of 0?

In [ ]:
# Create a pipeline that: 
# 1. Matches establishments with a hygiene score of 0
# 2. Groups the matches by Local Authority
# 3. Sorts the matches from highest to lowest

# Print the number of documents in the result

# Print the first 10 results


In [ ]:
# Convert the result to a Pandas DataFrame

# Display the number of rows in the DataFrame

# Display the first 10 rows of the DataFrame
